### Analyse de Subway_in  :
**Code Bokeh qui fonctionne pas bien. Il faudrait faire un checkbox en menu déroulant, pas tout afficher d'un coup, un bouton pour tout désactiver car 40 plot c'est long. Rajouter un 'hover', etc...**

In [ ]:
import numpy as np 

from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import Legend, LegendItem,HoverTool
from bokeh.palettes import Turbo256
from bokeh.layouts import row

# Personnal Import 
from utilities_DL import load_raw_data
from DL_class import DataSet
# ...

# Paths
folder_path = 'data/'

#Init and load data: 
H,W,D = 6,1,1
L = H+W+D 
step_ahead = 1

window_pred = np.arange(2*96)

file_name = 'subway_IN_interpol_neg_15_min_2019_2020.csv'



In [ ]:
# Load df, remove forbidden values: 
df,invalid_dates,time_step_per_hour = load_raw_data(folder_path,file_name,single_station = False)
dataset = DataSet(df,time_step_per_hour=time_step_per_hour, Weeks = W, Days = D, historical_len= H,step_ahead=step_ahead)
dataset.get_shift_between_set()   # get shift indice and shift date from the first element / between each dataset 
dataset.get_feature_vect()  # Build 'df_shifted'.
dataset.remove_forbidden_prediction(invalid_dates) # Build 'df_verif' , which is df_shifted without sequences which contains invalid date

# Find atypical Value. Check if we need to modify them.
valid_df = dataset.df[~(dataset.df.index.isin(invalid_dates))]
limit = 1500
all_plot = False

p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

# Stocker les renderers pour les ajouter à la légende
renderers = []

if all_plot:
    for i, column in enumerate(df.columns):
        x, y = df.index, df[column]
        color = Turbo256[int(256 / len(df.columns) * i)]  
        renderer = p.line(x, y, line_width=2, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
        renderer.visible = False
        renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(df.columns, renderers))])

else:

    for i,column in enumerate(valid_df):
        color = Turbo256[int(256 / len(df.columns) * i)]  
        sub_df = valid_df[valid_df[column] > limit]
        if not sub_df.empty:
            x,y = sub_df.index,sub_df[column]
            renderer = p.scatter(x, y, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
            renderers.append(renderer)

    x,y = dataset.df.index,dataset.df.quantile(0.9,axis = 1)
    renderer = p.line(x, y, line_width=2, color='black', alpha=0.8, legend_label='all_stations', muted_color=color, muted_alpha=0)
    renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(list(df.columns)+['all_stations'], renderers))])  


p.add_layout(legend, 'right')
p.legend.click_policy = "mute"
hover = HoverTool()
p.add_tools(hover)
output_notebook()
show(p) 

output_file("df_IN_outliers.html")
save(p)

In [ ]:
p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

column = 'Ampère Victor Hugo'
column = 'Bellecour'
x,y = dataset.df.index,dataset.df[column]

renderer = p.line(x, y, color='blue', alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)

output_notebook() 
show(p)

output_file(f"df_IN_{column}.html")
save(p)
